#### Define functions for random forest model

Random Forest models are ensemble machine learning algorithms widely used for classification and regression predictive modelling problems.  They are an ensemble of [decision tree algorithms](https://en.wikipedia.org/wiki/Decision_tree_learning) that can fit on slightly different examples of a single dataset in an attempt to produce a better overall result.  To validate these means, K-fold cross validation is often used.  This process involves taking random samples of the data and retraining the model so as not to overfit to one training sample.  Unfortunately, this method is not necessarily fit for modelling time series data as it can lead to overly optimistic models by sample future data and predicting on the past.

Jason Brownlee [documents a workflow](https://machinelearningmastery.com/random-forest-for-time-series-forecasting/) for Random Forest Time Series Forecasting which is used in this notebook to explore the feasibility of it for predicting ambient population. This will be discussed as the data is processed and the models run later.

A note on cross validation - Whilst the use of it for time series has been found to be feasible by Bergmeir et al (2018) by only using lagged versions of the outcome variable, it would be unsuitable for this project which seeks to use other explanatory variables alongside these.







In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
from plotly.subplots import make_subplots
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import lag_plot
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

import os

from source import *

# forecast monthly births with random forest

if not os.path.exists("../images"):
    os.mkdir("../images")

Load in data and define some functions

In [37]:
daily_footfall = pd.read_csv("../data/dailyfootfall.csv",parse_dates=['DateTime'],index_col='DateTime')

new_weather = pd.read_csv("../data/weatherdata.csv",parse_dates=['timestamp'],index_col='timestamp')
previous_weather = pd.read_csv("../data/overall_weather.csv",parse_dates=['date'],index_col='date',dayfirst=True)
bankhols = pd.read_csv('../data/ukbankholidays.csv',parse_dates=['ukbankhols'])
schoolterms = pd.read_csv('../data/schoolterms.csv',parse_dates=['date'],dayfirst=True,index_col='date',usecols=['schoolStatus','date'])

min_max_scaler = MinMaxScaler()

def validation_plot(y,yhat):
	# plot expected vs predicted
	fig = make_subplots()
	# Add traces
	fig.add_trace(
		go.Scatter(x=pd.Series(y).index, y=pd.Series(y), name="Expected",mode='lines',hovertemplate='%{y}'),
	)
	fig.add_trace(
		go.Scatter(x=pd.Series(y).index, y=pd.Series(yhat), name="Predicted",mode='lines',hovertemplate='%{y}'),
	)

	# Set x-axis title
	fig.update_xaxes(title_text="Time Series (Day)")

	# Set y-axes titles
	fig.update_yaxes(title_text="Daily Footfall")

	return fig

def movecol(df, cols_to_move=[], ref_col='', place='After'):

	cols = df.columns.tolist()
	if place == 'After':
		seg1 = cols[:list(cols).index(ref_col) + 1]
		seg2 = cols_to_move
	if place == 'Before':
		seg1 = cols[:list(cols).index(ref_col)]
		seg2 = cols_to_move + [ref_col]

	seg1 = [i for i in seg1 if i not in seg2]
	seg3 = [i for i in cols if i not in seg1 + seg2]

	return(df[seg1 + seg2 + seg3])

# Function to process date and recreate it in a different format
def create_date_format (col, old_time_format, new_time_format):
	col = pd.to_datetime(col, format = old_time_format )
	if type(col) == pd.core.series.Series:
		col = col.apply(lambda x: x.strftime(new_time_format))
	elif type(col) ==  pd.tslib.Timestamp:
		col = col.strftime(new_time_format)
	return col

def create_weather_predictors(dataf,weather_1=new_weather,weather_2=previous_weather):
	"""Create weather dataset and normalise values across the same range.

	PARAMETERS:
	  - weather 1 is a pandas dataframe containing combined weather data from the NCAS archive from 01/04/2017.
	  - weather 2 is a pandas dataframe containing weather data from a previous intern project up to 31/03/2017.
	"""

	weather_1 = weather_1.loc[weather_1.index >'2017-03-31']
	weather_1 = weather_1.resample("D").agg({'temp_°C':'mean',
												'rain_mm': 'sum',
												"wind_ms¯¹": 'mean'})
	weather_1 = weather_1.rename(columns={'temp_°C':'mean_temp',
												'rain_mm': 'rain',
												"wind_ms¯¹": 'wind_speed'})
	weather_2 = weather_2.drop(['abnormal_rain','high_temp',	'low_temp','high_wind'],axis=1)

	comb_weather = pd.concat([weather_2,weather_1])

	#Either interpolate missing data or drop.  Decide after consultations.
	#comb_weather = comb_weather.interpolate(method='time')
	comb_weather = comb_weather.dropna()

	dataf = dataf.merge(comb_weather,how='left',left_on=dataf.index,right_on=comb_weather.index).set_index('key_0')

	return dataf


def create_date_predictors(dataf):

	#dataf['year'] = pd.DatetimeIndex(dataf.index).year
	dataf['month'] = pd.DatetimeIndex(dataf.index).month_name()
	dataf['dayofweek'] = pd.DatetimeIndex(dataf.index).day_name()

	#dataf = pd.get_dummies(dataf, columns=['year'], drop_first=True, prefix='year')
	dataf = pd.get_dummies(dataf, columns=['month'], drop_first=True, prefix='month')
	dataf = pd.get_dummies(dataf, columns=['dayofweek'], drop_first=True, prefix='wday')

	return dataf

def create_holiday_predictors(dataf,bankholdf=bankhols,schooltermdf=schoolterms):
	bankholdf['bank_hols'] = 1

	schooltermdf['schoolholidays'] = np.where(schooltermdf['schoolStatus']=='Close',1,0)
	schooltermdf = schooltermdf.loc[schooltermdf.index >= '2008'].sort_index()
	schooltermdf = schooltermdf.asfreq('D')
	schooltermdf.ffill(inplace=True)

	dataf = dataf.merge(bankhols, left_on=dataf.index,right_on='ukbankhols',how='left')
	dataf = dataf.set_index('ukbankhols')
	dataf.bank_hols = dataf.bank_hols.fillna(0)

	dataf = dataf.merge(schooltermdf,how='left',left_on=dataf.index,right_on=schooltermdf.index).set_index('key_0').drop(['schoolStatus'],axis=1)

	return dataf

#The following workflow performs some data management to account for the dataframe requiring transformation into a numpy array to work with the walk forward validation code
def arrange_cols(dataf,n_in):
#Extract columns that need moving for walk forward validation later
	#cols_to_move = [col for col in dataf.iloc[:,0:7]] DEPRECATED, MAY NEED IN FutureWarning
	cols_to_move = []
	n_in = n_in+1
	for i in range(1,n_in):
		cols_to_move.append(f'var1(t-{i})')

	cols_to_move.append('var1(t)')
	#Identify reference column as last column
	ref_col = [col for col in dataf.iloc[:,-1:]][0]

	#Calls a function that moves specified columns to the end of the dataframe.
	dataf = movecol(dataf,
				 cols_to_move=cols_to_move,
				 ref_col=ref_col,
				 place='After')

	return dataf

def drop_na(dataf):

	dataf = dataf.dropna()

	return dataf

# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols,names = list(),list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data.iloc[:-n_test, :], data.iloc[-n_test:, :]

# fit an random forest model and make a one step prediction
def random_forest_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = RandomForestRegressor(n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]

# walk-forward validation for univariate data - NEEDS SOME WORK TO ADAPT FOR REFITTING SCALING TO TRAINING DATA AND APPLYING TO TEST
def walk_forward_validation(data, n_test, scalecols,n_in):
	print('Validation has started.  Please be patient, it may take a while and a message will be displayed when finished.')
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	#scale numerical data
	train.loc[:,scalecols] = min_max_scaler.fit_transform(train.loc[:,scalecols])
	test.loc[:,scalecols] = min_max_scaler.transform(test.loc[:,scalecols])
	#rearrange columns and record variable names in a dictionary
	train, test = arrange_cols(train,n_in), arrange_cols(test,n_in)
	#convert dataframes to numpy arrays
	train, test = train.values, test.values
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction
		yhat = random_forest_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		# summarize progress
		#print(i,'>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, -1], predictions

def create_prediction_data(yhatdf,test):
	yhatdf = pd.DataFrame(yhatdf)
	test = test.reset_index()

	yhatdf['datetime'] = test['key_0']
	yhatdf = yhatdf.set_index('datetime').rename(columns={0:'predicted'})
	yhatdf['roll_7_mean'] = yhatdf['predicted'].rolling(7).mean()

	return yhatdf

def daily_predicted_chart(yhat_list,finaldata):
	finaldata = finaldata.loc[finaldata.index >= '2018']
	fig = make_subplots()

	# Add traces
	fig.add_trace(
		go.Scatter(x=finaldata.index, y=finaldata['roll_7_mean'], name="Observed"),
	)

	fig.add_trace(
		go.Scatter(x=yhat_list[0].index, y=yhat_list[0]['roll_7_mean'], name="Predicted_1_lag"),
	)

	fig.add_trace(
		go.Scatter(x=yhat_list[1].index, y=yhat_list[1]['roll_7_mean'], name="Predicted_3_lag"),
	)

	fig.add_trace(
		go.Scatter(x=yhat_list[2].index, y=yhat_list[2]['roll_7_mean'], name="Predicted_7_lag"),
	)

	# Add figure title
	fig.update_layout(
		title_text="Rolling 7 day mean predictions"
	)

	# Set x-axis title
	fig.update_xaxes(title_text="DateTime")

	# Set y-axes titles
	fig.update_yaxes(title_text="Footfall")

	return fig

def weekly_predicted_chart(yhat_dataf,finaldata):
	finaldata = finaldata.loc[finaldata.index >= '2018']
	finaldata = finaldata.resample('W').agg({'var1(t)':'sum'})
	yhat_dataf = yhat_dataf.resample('W').agg({'predicted':'sum'})
	# Create figure with secondary y-axis
	fig = make_subplots()

	# Add traces
	fig.add_trace(
		go.Scatter(x=finaldata.index, y=finaldata['var1(t)'], name="yaxis data", connectgaps=False),
	)

	fig.add_trace(
		go.Scatter(x=yhat_dataf.index, y=yhat_dataf['predicted'], name="yaxis2 data",connectgaps=False),
	)

	# Add figure title
	fig.update_layout(
		title_text="Double Y Axis Example"
	)

	# Set x-axis title
	fig.update_xaxes(title_text="xaxis title")

	# Set y-axes titles
	fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
	fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

	return fig

def monthly_predicted_chart(yhat_dataf,finaldata):
	finaldata = finaldata.loc[finaldata.index >= '2019']
	finaldata = finaldata.resample('M').agg({'var1(t)':'sum'})
	yhat_dataf = yhat_dataf.resample('M').agg({'predicted':'sum'})
	# Create figure with secondary y-axis
	fig = make_subplots()

	# Add traces
	fig.add_trace(
		go.Scatter(x=finaldata.index, y=finaldata['var1(t)'], name="yaxis data", connectgaps=False),
	)

	fig.add_trace(
		go.Scatter(x=yhat_dataf.index, y=yhat_dataf['predicted'], name="yaxis2 data",connectgaps=False),
	)

	# Add figure title
	fig.update_layout(
		title_text="Double Y Axis Example"
	)

	# Set x-axis title
	fig.update_xaxes(title_text="xaxis title")

	# Set y-axes titles
	fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
	fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

	return fig

def create_data_cols(dataf):
	data_cols = [col for col in processed_data] #List containing column names from daily dataframe
	data_col_keys = list(range(len(data_cols))) # List containing integer positions of dataframe columns

	#Creates a dictionary of column names with integer keys representing position
	data_col_dict = dict(zip(data_col_keys, data_cols))

	return data_col_keys, data_cols


def create_importance_df(feature_import, datacols):
	importance = pd.DataFrame(feature_import)
	importance['feature_name'] = datacols[:-1]
	importance = importance.set_index('feature_name')
	importance = importance.rename(columns={0:'feat_importance'}).sort_values(by='feat_importance',ascending=False)

	return importance


### Random Forest Models for Footfall

The following code blocks generate two Random Forest Models.

The first is trained on all available data up to the most recent (currently April 2021).  It includes lockdown specific conditions and other explanatory variables such as lagged outcomes, weather and time series specific inputs such as day of the week or month of the year. This could quantify how important lockdown variables were in explaining the changes in footfall that occur during periods of lockdown.

The second model is trained just on data up to the beginning of lockdown and ignore the conditions, given that it would be predicting 'business as usual'.  Essentially they're pretending the pandemic never happened so only need to consider the elements that were found to be important in previous modelling of footfall data by Molly Asher.

This approach explores several avenues.  First, it covers quantifying the changes and explains general footfall and then cover making predictions.  It would be useful to have the model with lockdown variables available for future prediction if they turn out to have good explanatory power compared with the usual predictors.

#### Model #1 - Explanation of lockdown variable strength when considering changes in footfall during pandemic period.

To create the models, data is processed as follows:

- Data is transformed into a supervised learning problem, with various time lags created using pandas.shift().  These lags become X input variables, with the original counts becoming a y output.  Each row in the dataset contains however many time shifts you've calculated as X inputs, plus any additional variables you'd like to provide explanatory power
- Creation of lockdown predictors to represent the different parts of society that were closed during lockdowns since March 2020. They cover things such as non-essential retail, hospitality, entertainment, sport/leisure, weddings, education and maximum group sizes.  These are a mix of label encoded and dummy variables.  Some of these might be subject to change depending on model performance
- Creation of Date predictors, time series specific input variables such as day of week, month of year or even whether the day is a weekend or not
- Creation of Holiday predictors that represent whether the day falls on a bank or school holiday
- Creation of Weather predictors that consist of mean daily temperature, total daily rainfall and mean daily wind speed
- Removal of missing values
- Arrange columns so that the observed y variable is the last in the dataframe.  This is required for walk-forward validation later on

This is undertaken for each model, with some changes that will be discussed in the relevant markdown section.

#### Walk Forward Validation

The walk-forward validation method iterates through a test set explicitly specified as being towards the end of the data sequence using the explanatory variables generated to predict values, comparing against expected and providing an MAE.  Once all the parameters have been set and model finalised, then variable importance can be extracted and predictions on out of sample data can be made.

<b>Please note that this method is relatively computationally expensive</b> as it loops through each row and retrains the model to include additional footfall values.  Only run if you want to see the outputs.  It is not required to see the results of the actual model.

In [34]:
#set number of time lagged variables
n_in = 1
# transform the time series data into supervised learning and add additional explanatory variables
processed_data = (daily_footfall
				  .pipe(start_pipeline)
				  .pipe(series_to_supervised,n_in=n_in)
				  .pipe(create_lockdown_predictors)
				  .pipe(create_date_predictors)
				  .pipe(create_holiday_predictors)
				  .pipe(create_weather_predictors)
				  .pipe(drop_na)
                  .pipe(arrange_cols,n_in=n_in))

cols_to_scale = ['mean_temp', 'wind_speed', 'rain', 'hosp_indoor', 'hosp_outdoor', 'hotels', 'ent_indoor', 'ent_outdoor',
				 'weddings', 'self_acc', 'sport_lei_indoor', 'sport_lei_outdoor', 'non_ess_retail', 'prim_sch',
				 'sec_sch', 'uni_campus', 'outdoor_grp_public', 'outdoor_grp_private', 'indoor_grp', 'eat_out']

#variables to count how many days fall after 23rd March 2020
lockdown_days = len(processed_data.loc[processed_data.index >= "2020-03-23"])
#variable to count how many days fall after 1st January 2021
model1_test_days = len(processed_data.loc[processed_data.index >= "2021-01-01"])

#evaluate model using walk forward validation.  The default days are set at 12 for now.  To set a custom number, please change n_test.  To use all days after 1st January 2021 please set n_test to model1_test_days.  NOTE THIS WILL TAKE A LONG TIME TO VALIDATE.
mae, y, yhat = walk_forward_validation(processed_data, 12, cols_to_scale, n_in)
print('Validation has finished.  The MAE is: %.3f' % mae)

validation_plot(y,yhat).show()

validation_plot(y,yhat).write_image("../images/Model1Validation.png")

Validation has started.  Please be patient, it may take a while and a message will be displayed when finished.


C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Validation has finished.  The MAE is: 4613.743


The validation seems to show that the model works fairly well with the shifts in lockdown variables if validating on the period from 1st January 2021.  It's computationally quite expensive to validate on too much data but it will show that the model can follow the general trend of changes.

There are a few reasons why it might take so much time.  The first is that it's a large dataset.  Reducing the amount of training data might help it run quicker.

Furthermore, reducing the dimensionality of the data might also help.

#### Variable Importance

The following code repeats the data processing pipeline, fits the RandomForest model from above and extracts variable importance.

In [30]:
n_in=1
processed_data = (daily_footfall
				  .pipe(start_pipeline)
				  .pipe(series_to_supervised,n_in=n_in)
				  .pipe(create_lockdown_predictors)
				  .pipe(create_date_predictors)
				  .pipe(create_holiday_predictors)
				  .pipe(create_weather_predictors)
				  .pipe(drop_na)
                  .pipe(arrange_cols,n_in=n_in))

cols_to_scale = ['mean_temp', 'wind_speed', 'rain', 'hosp_indoor', 'hosp_outdoor', 'hotels', 'ent_indoor', 'ent_outdoor',
				 'weddings', 'self_acc', 'sport_lei_indoor', 'sport_lei_outdoor', 'non_ess_retail', 'prim_sch',
				 'sec_sch', 'uni_campus', 'outdoor_grp_public', 'outdoor_grp_private', 'indoor_grp', 'eat_out']

values = processed_data.values

data_cols_pos, data_cols = create_data_cols(processed_data)

#split into input and output columns
trainX, trainy = processed_data.iloc[:, :-1], processed_data.iloc[:, -1]
trainX.loc[:,cols_to_scale] = min_max_scaler.fit_transform(trainX.loc[:,cols_to_scale])
#fit model
model = RandomForestRegressor(n_estimators=1000)
model.fit(trainX, trainy)
#extract feature importance
importance = create_importance_df(model.feature_importances_,data_cols)


fig = px.bar(importance, orientation='v')
fig.update_xaxes(nticks=30,tickfont=dict(size=8))
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.write_image("../images/Model1Importance.png")
importance

,feat_importance
feature_name,
ent_indoor,0.416400
var1(t-1),0.177456
wday_Saturday,0.160759
wday_Sunday,0.083372
mean_temp,0.028878
wind_speed,0.022379
non_ess_retail,0.018191
rain,0.016714
bank_hols,0.014571


Feature importance is a useful measure of assessing what predictors have strong power in predicting the outcome variable.  They do not necessarily explain why footfall changes at specific points but are a useful indicator of what is dominant within the model.  It may be that this can be used to reduce the dimensionality in the model for future iterations.

At this stage the model only uses 1 time lagged variable as an input.  This is important as the more of these are introduced could potentially increase the predictive power of the model, however they might come to dominate the feature importance, pushing others down and rendering them relatively useless.  The other problem is that these time lagged variables are large numerical values compared to the other scaled variables.  Scaling them would most likely remove their ability to offer much predictive value within the model so they are left as they are.  Unfortunately this means that they are likely to dominate in whatever model they are included in as shown in the example below which sets 7 lagged variables.

In [5]:
n_in=7
processed_data = (daily_footfall
				  .pipe(start_pipeline)
				  .pipe(series_to_supervised,n_in=n_in)
				  .pipe(create_lockdown_predictors)
				  .pipe(create_date_predictors)
				  .pipe(create_holiday_predictors)
				  .pipe(create_weather_predictors)
				  .pipe(drop_na)
                  .pipe(arrange_cols,n_in=n_in))

cols_to_scale = ['mean_temp', 'wind_speed', 'rain', 'hosp_indoor', 'hosp_outdoor', 'hotels', 'ent_indoor', 'ent_outdoor',
				 'weddings', 'self_acc', 'sport_lei_indoor', 'sport_lei_outdoor', 'non_ess_retail', 'prim_sch',
				 'sec_sch', 'uni_campus', 'outdoor_grp_public', 'outdoor_grp_private', 'indoor_grp', 'eat_out']

values = processed_data.values

data_cols_pos, data_cols = create_data_cols(processed_data)

#split into input and output columns
trainX, trainy = processed_data.iloc[:, :-1], processed_data.iloc[:, -1]
trainX.loc[:,cols_to_scale] = min_max_scaler.fit_transform(trainX.loc[:,cols_to_scale])
#fit model
model = RandomForestRegressor(n_estimators=1000)
model.fit(trainX, trainy)
#extract feature importance
importance = create_importance_df(model.feature_importances_,data_cols)


fig = px.bar(importance, orientation='v')
fig.update_xaxes(nticks=30,tickfont=dict(size=8))
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.write_image("../images/Model1Importance.png")
importance

,feat_importance
feature_name,
var1(t-7),0.687824
var1(t-1),0.118542
var1(t-2),0.036974
wday_Saturday,0.024408
var1(t-6),0.018516
var1(t-3),0.014195
wday_Sunday,0.011894
var1(t-4),0.011888
rain,0.010998


This time around the time lagged variables dominate the model, not really helping to explain how different aspects of lockdown might have impacted footfall throughout the pandemic period.

Model #2 - Prediction of business as usual

The following code repeats the initial data processing undertaken on the previous models, however omits the creation of lockdown variables.  This is because the idea is just to predict what footfall might have been like if the pandemic and lockdown measures didn't happen.  The initial pipeline steps through creating a supervised learning dataset and relevant footfall predictors.  Several tests have been run using different processes.

This first block drops missing values from the data altogether, regardless of what columns these occur in.  In most checks, missing values only really occurred in weather related variables as data were not available.

In [71]:
lags = [1,3,7]
mae_list = []
validation_df = pd.DataFrame()
for lag in lags:
	n_in=lag
	# transform the time series data into supervised learning and add additional explanatory variables.  Filter to exclude period where lockdowns were in effect.
	processed_data = (daily_footfall
					  .pipe(start_pipeline)
					  .pipe(series_to_supervised,n_in=n_in)
					  .pipe(create_date_predictors)
					  .pipe(create_holiday_predictors)
					  .pipe(create_weather_predictors)
					  .pipe(drop_na)
					  .pipe(arrange_cols,n_in=n_in))

	#create list of columns that need scaling
	cols_to_scale = ['mean_temp', 'wind_speed', 'rain']

	#split dataset based on when first measures to reduce social contact were put in place.
	prelockdown_data = processed_data.loc[processed_data.index < "2020-03-16"]
	lockdown_data = processed_data.loc[processed_data.index >= "2020-03-16"]

	#evaluate model for prediction of business as usual during lockdown period.
	mae, y, yhat = walk_forward_validation(prelockdown_data, 30, cols_to_scale,n_in=n_in)
	mae_list.append(mae)
	validation_df[f'timelag_{lag}'] = yhat

validation_df['y'] = y
fig = px.line(validation_df)
fig.show()
# plot expected vs predicted
# validation_plot(y,yhat).show()
# validation_plot(y,yhat).write_image("../images/Model2Validation.png")

Validation has started.  Please be patient, it may take a while and a message will be displayed when finished.


C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Validation has started.  Please be patient, it may take a while and a message will be displayed when finished.


C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Validation has started.  Please be patient, it may take a while and a message will be displayed when finished.


C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\envs\amb_pop\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [72]:
maedf = pd.DataFrame(mae_list)
maedf

,0
0,14327.091433
1,8990.093367
2,9104.665333


The validation above gives an indication of how accurate the model is on this run.  There may be differences each time it's run so the code could be developed to run a number of validations and see if the MAE changes much.  All forecasting is tricky, particularly with time series data, but the model seems to do a fairly good job at getting close to the predicted values for each day of the validation set.

The next stage is to use the model above to make predictions on an 'out of sample' test set.  In this scenario, the data we want to predict on is everything after and including 16th March 2020, when large changes in footfall started to be seen after social distancing announcements.

The same data processing pipeline is used, with train and test data split using the 16th March 2020 point.

In [66]:
importance_list = []
yhat_list = []

lags = [1,3,7]
for lag in lags:
	n_in=lag
	# transform the time series data into supervised learning and add additional explanatory variables.  Filter to exclude period where lockdowns were in effect.
	processed_data = (daily_footfall
					  .pipe(start_pipeline)
					  .pipe(series_to_supervised,n_in=n_in)
					  .pipe(create_date_predictors)
					  .pipe(create_holiday_predictors)
					  .pipe(create_weather_predictors)
					  .pipe(drop_na)
					  .pipe(arrange_cols,n_in=n_in))

	#Set columns to scale
	cols_to_scale = ['mean_temp', 'wind_speed', 'rain']

	#Create data col files to allow prediction outputs to be joined back to datetime later.
	data_col_pos, data_cols = create_data_cols(processed_data)

	#Split into training and testing datasets
	train = processed_data.loc[processed_data.index < "2020-03-16"]
	test = processed_data.iloc[processed_data.index >= "2020-03-16"]

	#split training data into input and output columns
	trainX, trainy = train.iloc[:, :-1], train.iloc[:, -1]
	#Fit and apply scaling to training data
	trainX.loc[:,cols_to_scale] = min_max_scaler.fit_transform(trainX.loc[:,cols_to_scale])

	#fit model
	model = RandomForestRegressor(n_estimators=1000)
	model.fit(trainX, trainy)

	#split test data into input and output columns
	testX, testy = test.iloc[:, :-1], test.iloc[:, -1]
	#Apply scaler to test data
	testX.loc[:,cols_to_scale] = min_max_scaler.transform(testX.loc[:,cols_to_scale])

	#extract feature importance and join back to variable names
	importance = create_importance_df(model.feature_importances_, data_cols)
	importance_list.append(importance)



	yhat_list.append(create_prediction_data(model.predict(testX),test))

processed_data['roll_7_mean'] = processed_data['var1(t)'].rolling(7).mean()
prediction_fig = daily_predicted_chart(yhat_list,processed_data)
importance_fig = []

for df,lag in zip(importance_list,lags):
	fig = df.head(10).plot.bar(color=df.head(10).index,
										   title=f"Model 2 Variable Importance (Top 10) - {lag} time lagged",
										   labels={'value': 'Variable Importance',
												   'feature_name': 'Model Feature'})
	fig.update_layout(showlegend=False,
					  title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
	importance_fig.append(fig)

Plot predictions chart for 1,3 and 7 time lagged variables

In [62]:
fig = prediction_fig
fig.show()
fig.write_image(f"../images/Model2PredictionA.svg")

Plot importance chart for 1 time lagged variable

In [67]:
#plot importance chart
importance_fig[0].show()
importance_fig[0].write_image(f"../images/Model2ImportanceA1.svg")

Plot importance chart for 3 time lagged variables

In [68]:
#plot importance chart
importance_fig[1].show()
importance_fig[1].write_image(f"../images/Model2ImportanceA3.svg")

Plot importance chart for 7 time lagged variables

In [69]:
#plot importance chart
importance_fig[2].show()
importance_fig[2].write_image(f"../images/Model2ImportanceA7.svg")

There's a lot going on in those charts, but to summarise:
- Introduction of more than one time lagged variable seems to throw the predictions off in a big way.
- At 7 variables, the predictions mirror more what actually happened due to lockdown rather than what might have been.

There's much less variation in the predictions than actual values from previous years.  There's almost an immediate drop after 16th March 2020 which would go against the trend seen in the data previously.  Maybe this is because seasonality and trends haven't been removed in pre-processing.  It's difficult to quantify any changes right now as I'm not particularly confident this works well.  There's also horrible artifacts in the weekly and monthly resamples caused by dropping entire weeks/months where weather data is missing.

It might be worth looking at variable importance, removing some of the less powerful ones and potentially imputing some weather data.

The following blocks of code rerun the analysis but with a process to impute weather data.  Note that as it stands the method isn't as robust as it could be.  Really the imputation should be done after splitting the train/test data for validation purposes, however I tried to do so and then rejoin the sets back together.  For some reason Python had problems with the test_train_split function within the validation so I abandoned this and just carried the imputing out as one dataset.  This means that there could be some data leakage but I'm happy to accept that for now as the scaling is still done after the split.

In [ ]:
#transform the time series data into supervised learning and add additional explanatory variables.  Filter to exclude period where lockdowns were in effect.
processed_data = (daily_footfall
				  .pipe(start_pipeline)
				  .pipe(series_to_supervised,n_in=6)
				  .pipe(create_date_predictors)
				  .pipe(create_holiday_predictors)
				  .pipe(create_weather_predictors)
                  .pipe(arrange_cols))

#lists of columns to scale and impute (may or may not be different)
cols_to_scale = ['mean_temp', 'wind_speed', 'rain']
cols_to_impute = ['mean_temp', 'wind_speed', 'rain']

#set processed data as pre lockdown period
processed_data = processed_data.loc[processed_data.index < "2020-03-16"]
#interpolate columns with missing values using a time based method.
processed_data = processed_data.interpolate(method='time')
#drop missing values
processed_data = processed_data.dropna()

#DEPRECATED CODE - initially split, impute and then rejoin but it was causing an indexing error within the walk forward validation method.
# test_rows = 30
# train, test_rows = train_test_split(processed_data,test_rows)
#
# train.loc[:,cols_to_impute], test_rows.loc[:,cols_to_impute] = train.loc[:,cols_to_impute].interpolate(method='time'), test_rows.loc[:,cols_to_impute].interpolate(method='time')
# train,test_rows = train.dropna(),test_rows.dropna()
#
# prelockdown_data = pd.concat([train,test_rows],axis=0)

In [ ]:
#evaluate model for prediction of business as usual during lockdown period.
mae, y, yhat = walk_forward_validation(processed_data, 30, cols_to_scale)
print('MAE: %.3f' % mae)
# plot expected vs predicted
validation_plot(y,yhat).show()
validation_plot(y,yhat).write_image("../images/Model2ValidationA.png")


The validation above isn't that much different from the previous round.  The imputing hasn't necessarily improved the predictions of the model on a test set.  Let's see what happens when predicting out of sample.

In [ ]:
# transform the time series data into supervised learning and add additional explanatory variables.  Filter to exclude period where lockdowns were in effect.
processed_data = (daily_footfall
				  .pipe(start_pipeline)
				  .pipe(series_to_supervised,n_in=6)
				  .pipe(create_date_predictors)
				  .pipe(create_holiday_predictors)
				  .pipe(create_weather_predictors)
                  .pipe(arrange_cols))

cols_to_scale = ['mean_temp', 'wind_speed', 'rain']
cols_to_impute = ['mean_temp', 'wind_speed', 'rain']


train = processed_data.iloc[processed_data.index < "2020-03-16"]
test = processed_data.iloc[processed_data.index >= "2020-03-16"]

#Create data col files to allow prediction outputs to be joined back to datetime later.
data_col_pos, data_cols = create_data_cols(processed_data)


#Impute missing weather data
train.loc[:,cols_to_impute] = train.loc[:,cols_to_impute].interpolate(method='time')
#drop missing rows
train = train.dropna()
#arrange columns so that y is last col
train = arrange_cols(train)
#split into input and output columns
trainX, trainy = train.iloc[:, :-1], train.iloc[:, -1]
#Fit and apply scaling to train dataset
trainX.loc[:,cols_to_scale] = min_max_scaler.fit_transform(trainX.loc[:,cols_to_scale])

#fit model
model = RandomForestRegressor(n_estimators=1000)
model.fit(trainX, trainy)

#Create test data
test.loc[:,cols_to_impute] = test.loc[:,cols_to_impute].interpolate(method='time')
test = test.dropna()
test = arrange_cols(test)
testX, testy = test.iloc[:, :-1], test.iloc[:, -1]
testX.loc[:,cols_to_scale] = min_max_scaler.transform(testX.loc[:,cols_to_scale])

#extract feature importance
importance = create_importance_df(model.feature_importances_,data_cols)
#importance_trans = importance.transpose()
fig = importance.plot.bar()
fig.show()
fig.write_image("../images/Model2Importance.png")

In [12]:
yhat = create_prediction_data(model.predict(testX),test)
fulldata = pd.concat([train,test], axis=0)

funcs = [daily_predicted_chart,weekly_predicted_chart,monthly_predicted_chart]
figs = []
fulldata['roll_7_mean'] = fulldata['var1(t)'].rolling(7).mean()
for func in funcs:
	fig = func(yhat,fulldata)
	fig.write_image(f"../images/Model1Validation.png")
	figs.append(fig)

figs[0].show()

In [ ]:
figs[1].show()

In [ ]:
figs[2].show()

Spent quite a bit of time on this.  Due to the way in which the validation has been set up, it's difficult to fit it into the 'conventional' scikitlearn pipeline and evaluate the hyperparameters so I'll leave those there for now.  Tweaking them might result in different predictions but it might just be that this model isn't the best/will take a lot of effort to get something usable.

I've spent a bit of time tidying up the code above to run the preprocessing through pipes, but not necessarily the SciKit learn pipelines used explicitly for modelling.  Due to the transformation of several variables into standardised/normalised ranges prior to splitting the data, I'll need to come back to this and adjust it so that only the variables are created at this point and any adjustments (such as scaling or imputing) are undertaken at the correct time.

The scaling is still causing problems but I think I've found a solution:
    - After seeding the history list, pull the code to convert into an array and split off the X and y variables.
    - Within the loop, split off test X and y.  Don't isolate the single row in the loop, instead do it inside the random forest function.
    - Instead of feeding history into the random forest function, feed in train X and y as they are.
    - Fit the model inside the random forest function and apply scaling to test X.  isolate the single row the loop is currently on and then predict on that.
    - After appending the last observation to the history, turn it into an array, split off train X and scale it once again.
    - Return to the start of the loop with a new, scaled train X and repeat, refitting the model on this and rescaling test X after.

I think that flow of logic will allow me to train on an initial set of data and then refit and reapply as the model is fit over and over again.  The only real problem is identifying the variables as they will be in matrix/array form, but I should have a list of variables and their column indexes stored to identify this.  The problem with this is that scaling them may inadvertently move them around in the dataset.  I should test this outside the loop before implementing.  If it does and they're always consistently moved to the end, it might just be a case of manually moving the relevant variables to the end of the train/test X datasets so that they can be accessed with just a column slice.

I'm not going to worry too much about this now at the end of the week.  I'd rather get the models set up and running with the scaling as best as it can be.  If the model is doing well in validation and can make predictions that seem fairly sensible then that'll be good enough for now.  Moving onto ARIMA and Neural Networks for comparison would be a better use of my time and I can come back and fix/tinker with any known issues later (or leave them for somebody else to deal with).




